# Task 3: 构建与索引向量知识库

本Notebook演示如何使用Task 3的功能将Task 2生成的向量化知识块构建成可进行高效相似度搜索的向量知识库。

## 主要功能

1. **数据加载**: 读取`vectorized_chunks.jsonl`文件
2. **索引构建**: 使用FAISS构建高效的向量索引
3. **GPU加速**: 自动检测并使用GPU加速索引构建
4. **文件保存**: 生成`knowledge_base.index`和`chunk_metadata.pkl`文件

## 输出文件

- `output/knowledge_base.index`: FAISS二进制索引文件
- `output/chunk_metadata.pkl`: 元数据映射文件

## 1. 环境检查与依赖导入

In [ ]:
import os
import json
import pickle
import numpy as np

# 检查必要的库
try:
    import faiss
    print(f"✅ FAISS版本: {faiss.__version__}")
    
    # 检查GPU支持
    gpu_count = faiss.get_num_gpus()
    if gpu_count > 0:
        print(f"🚀 检测到 {gpu_count} 个GPU，将使用GPU加速")
    else:
        print("💻 未检测到GPU，将使用CPU构建索引")
        
except ImportError:
    print("❌ 请先安装faiss-gpu: pip install faiss-gpu")

# 检查输入文件
input_file = "output/vectorized_chunks.jsonl"
if os.path.exists(input_file):
    file_size = os.path.getsize(input_file) / (1024 * 1024)  # MB
    print(f"✅ 输入文件存在: {input_file} ({file_size:.2f} MB)")
else:
    print(f"❌ 输入文件不存在: {input_file}")
    print("请先运行Task 2生成向量化数据文件")

## 2. 导入构建器类

In [ ]:
from build_index import VectorKnowledgeBaseBuilder

# 创建构建器实例
builder = VectorKnowledgeBaseBuilder(
    input_file="output/vectorized_chunks.jsonl",
    output_dir="output"
)

print("构建器已初始化")

## 3. 执行向量知识库构建

In [ ]:
# 执行完整的构建流程
try:
    print("开始构建向量知识库...")
    builder.build()
    print("\n🎉 向量知识库构建完成!")
    
except Exception as e:
    print(f"❌ 构建失败: {e}")
    import traceback
    traceback.print_exc()

## 4. 验证输出文件

In [ ]:
# 验证输出文件
index_file = "output/knowledge_base.index"
metadata_file = "output/chunk_metadata.pkl"

print("=== 输出文件验证 ===")

# 检查文件存在性
if os.path.exists(index_file):
    size_mb = os.path.getsize(index_file) / (1024 * 1024)
    print(f"✅ 索引文件: {index_file} ({size_mb:.2f} MB)")
else:
    print(f"❌ 索引文件不存在: {index_file}")

if os.path.exists(metadata_file):
    size_mb = os.path.getsize(metadata_file) / (1024 * 1024)
    print(f"✅ 元数据文件: {metadata_file} ({size_mb:.2f} MB)")
else:
    print(f"❌ 元数据文件不存在: {metadata_file}")


## 5. 测试搜索功能

In [ ]:
# 测试搜索功能
try:
    import faiss
    import pickle
    
    # 加载索引和元数据
    index = faiss.read_index("output/knowledge_base.index")
    with open("output/chunk_metadata.pkl", "rb") as f:
        metadata_list = pickle.load(f)
    
    print(f"📊 索引信息:")
    print(f"   向量数量: {index.ntotal:,}")
    print(f"   向量维度: {index.d}")
    print(f"   元数据条目数: {len(metadata_list):,}")
    
    # 测试搜索
    if index.ntotal > 0:
        query_vector = index.reconstruct(0).reshape(1, -1)
        distances, indices = index.search(query_vector, k=3)
        
        print(f"\n🔍 搜索测试结果:")
        for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
            metadata = metadata_list[idx]
            print(f"  [{i+1}] 距离: {dist:.6f}, chunk_id: {metadata['chunk_id']}")
        
        if distances[0][0] < 1e-6:
            print("✅ 搜索功能正常")
        else:
            print("⚠️ 搜索结果异常")
    
except Exception as e:
    print(f"❌ 搜索测试失败: {e}")

## 总结

Task 3成功完成了以下功能:

✅ **数据加载**: 从`vectorized_chunks.jsonl`加载向量和元数据
✅ **索引构建**: 使用FAISS IndexFlatL2构建高效向量索引
✅ **GPU加速**: 自动检测并使用GPU加速构建过程
✅ **数据对齐**: 确保向量索引与元数据映射的严格对应
✅ **持久化存储**: 生成可重用的索引和元数据文件
✅ **搜索功能**: 支持高效的相似度搜索

生成的向量知识库现在可以用于RAG系统的快速检索阶段。